In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
from IPython.display import clear_output

import quick_pp.las_handler as las
from quick_pp.lithology.multi_mineral import MultiMineral
from quick_pp.plotter.plotter import plotly_log
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold

In [ ]:
with open(r'data\01_raw\36_7-3.las', 'rb') as f:
    df, header = las.read_las_files([f])
df['RT'] = df['RDEP']

# Mask outside threshold
df = mask_outside_threshold(df, True)

# Flag bad hole
df = badhole_flagging(df) if 'CALI' in df.columns else df
clear_output()

In [ ]:
from collections import OrderedDict
import numpy as np
from quick_pp.plotter.well_log_config import COLOR_DICT

TRACE_DEFS = OrderedDict(
    GR=dict(
        track=1,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['GR']}
    ),
    RT=dict(
        track=2,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['RT']}
    ),
    RHOB=dict(
        track=3,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_width': 1, 'line_color': COLOR_DICT['RHOB']}
    ),
    DTC=dict(
        track=4,
        secondary_y=False,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['DTC']}
    ),
    VCLD=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=False,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tozerox', 'fillpattern_bgcolor': COLOR_DICT['VCLD'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '-',
            'fillpattern_size': 2, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    NPHI=dict(
        track=3,
        secondary_y=True,
        hide_xaxis=False,
        style={'line_dash': 'dot', 'line_width': 1, 'line_color': COLOR_DICT['NPHI']}
    ),
    VSAND=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VSAND'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VSILT=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VSILT'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VCALC=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VCALC'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '.',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.1, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
    VDOLO=dict(
        track=5,
        secondary_y=False,
        hide_xaxis=True,
        style={
            'line_width': .5, 'line_color': 'black', 'fill': 'tonextx', 'fillpattern_bgcolor': COLOR_DICT['VDOLO'],
            'fillpattern_fgcolor': '#000000', 'fillpattern_fillmode': 'replace', 'fillpattern_shape': '-',
            'fillpattern_size': 3, 'fillpattern_solidity': 0.3, 'stackgroup': 'litho', 'orientation': 'h'
        }
    ),
)

In [ ]:
font_size = 8
XAXIS_DEFS = {
    'GR': {
        'title': 'GR',
        'titlefont': {'color': COLOR_DICT['GR'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['GR'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'dtick': 40, 'range': [0, 200], 'type': 'linear', 'zeroline': False
    },
    'RT': {
        'title': 'RT',
        'titlefont': {'color': COLOR_DICT['RT'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['RT'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [np.log10(.2), np.log10(2000)], 'type': 'log',
        'tickmode': 'array', 'tickvals': np.geomspace(0.2, 2000, 5), 'tickangle': -90, 'minor_showgrid': True
    },
    'RHOB': {
        'title': 'RHOB',
        'titlefont': {'color': COLOR_DICT['RHOB'], 'size': font_size},
        'tickformat': ".2f", 'tick0': 1.95, 'dtick': 0.2, 'tickangle': -90,
        'tickfont': {'color': COLOR_DICT['RHOB'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'range': [1.95, 2.95], 'type': 'linear'
    },
    'NPHI': {
        'title': 'NPHI',
        'titlefont': {'color': COLOR_DICT['NPHI'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['NPHI'], 'size': font_size}, 'zeroline': False,
        'side': 'top', 'anchor': 'free', 'position': .89, 'title_standoff': .1, 'overlaying': 'x3',
        'tickformat': ".2f", 'tick0': -.15, 'dtick': 0.12, 'range': [.45, -.15], 'type': 'linear', 'tickangle': -90
    },
    'VCLD': {
        'title': 'LITHOLOGY',
        'titlefont': {'color': COLOR_DICT['VSHALE'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['VSHALE'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85, 'title_standoff': .1,
        'dtick': 0.1, 'range': [0, 1], 'type': 'linear', 'zeroline': False
    },
    'DTC': {
        'title': 'DTC',
        'titlefont': {'color': COLOR_DICT['DTC'], 'size': font_size},
        'tickfont': {'color': COLOR_DICT['DTC'], 'size': font_size},
        'side': 'top', 'anchor': 'free', 'position': .85,
        'title_standoff': .1, 'dtick': 20, 'range': [30, 150], 'type': 'linear', 'zeroline': False
    }
}

In [ ]:
neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = MultiMineral().estimate_lithology(
    neu_den_df['GR'], neu_den_df['NPHI'], neu_den_df['RHOB'], neu_den_df['DTC'], neu_den_df['PEF'])
df_ssc_model = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLD': vol_shale, 'VDOLO': vol_dolomite, 'VCALC': vol_calcite},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
fig.write_html('plot_all curves.html')

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
neu_den_df[['DTC', 'PEF']] = np.nan
vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = MultiMineral().estimate_lithology(
    neu_den_df['GR'], neu_den_df['NPHI'], neu_den_df['RHOB'], neu_den_df['DTC'], neu_den_df['PEF'])
df_ssc_model = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLD': vol_shale, 'VDOLO': vol_dolomite, 'VCALC': vol_calcite},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
fig.write_html('plot_basic log curves.html')

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
neu_den_df[['DTC']] = np.nan
vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = MultiMineral().estimate_lithology(
    neu_den_df['GR'], neu_den_df['NPHI'], neu_den_df['RHOB'], neu_den_df['DTC'], neu_den_df['PEF'])
df_ssc_model = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLD': vol_shale, 'VDOLO': vol_dolomite, 'VCALC': vol_calcite},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
fig.write_html('plot_no dtc log curves.html')

In [ ]:
import numpy as np

neu_den_df = df[['DEPTH', 'GR', 'NPHI', 'RHOB', 'DTC', 'PEF']].dropna()
neu_den_df[['PEF']] = np.nan
vol_quartz, vol_calcite, vol_dolomite, vol_shale, vol_mud = MultiMineral().estimate_lithology(
    neu_den_df['GR'], neu_den_df['NPHI'], neu_den_df['RHOB'], neu_den_df['DTC'], neu_den_df['PEF'])
df_ssc_model = pd.DataFrame(
    {'VSAND': vol_quartz, 'VCLD': vol_shale, 'VDOLO': vol_dolomite, 'VCALC': vol_calcite},
    index=neu_den_df.DEPTH
)
df_ssc = df.merge(df_ssc_model, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_ssc, trace_defs=TRACE_DEFS, xaxis_defs=XAXIS_DEFS)
fig.show(config=dict(scrollZoom=True))
fig.write_html('plot_no pef log curves.html')